# Multi-factor ANOVAs to support PPI analyses

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

root = '/Users/haileytrier/Downloads/Trier_et_al_2023_code/data/'

def getPeakVals(df, reg_name, search_window, ROI, seed):
    '''
    Extract peak values from the spreadsheet for a particular regressor within a search window.
    df: data from csv file.
    reg_name: name of regressor desired.
    '''
    cols = [x for x in df.columns if 'Peak_vals' in x]
    peaks = df.loc[(df.Regressor==reg_name)&(df.Peak_search_window==search_window)&(df.ROI==ROI)&(df.Seed==seed),cols].values[0].copy()
    pval = df.loc[(df.Regressor==reg_name)&(df.Peak_search_window==search_window)&(df.ROI==ROI)&(df.Seed==seed),'Uncorrected_pvalue'].values[0].copy()
    sig = True if pval < 0.05 else False
    return peaks, sig

def getPeakVals_noSeed(df, reg_name, search_window, ROI):
    '''
    Extract peak values from the spreadsheet for a particular regressor within a search window.
    df: data from csv file.
    reg_name: name of regressor desired.
    '''
    cols = [x for x in df.columns if 'Peak_vals' in x]
    peaks = df.loc[(df.Regressor==reg_name)&(df.Peak_search_window==search_window)&(df.ROI==ROI),cols].values[0].copy()
    pval = df.loc[(df.Regressor==reg_name)&(df.Peak_search_window==search_window)&(df.ROI==ROI),'Uncorrected_pvalue'].values[0].copy()
    sig = True if pval < 0.05 else False
    return peaks, sig

def addFactorsToDataframe(df, seed:str=np.nan, ROI:str=np.nan, peaks:float=np.nan, 
                          win:str=np.nan, success:str=np.nan, phase:str=np.nan, action:str=np.nan):
    '''
    Add ROI and seed as factors in the dataframe along with peak values.
    '''
    df = pd.concat([df,pd.DataFrame(data={
        'Seed':[seed for x in range(len(peaks))],
        'ROI':[ROI for x in range(len(peaks))],
        'Peaks':peaks,
        'Search_window':[win for x in range(len(peaks))],
        'Success':success,
        'Phase':phase,
        'Action':action
    })])
    df = df.reset_index(drop=True)
    return df.copy()

# Fig. 4Ai

2-way ANOVA testing the effects of cortical region (ACC/AI) and sub-cortical region (Hb/DRN) on the extent to which check switches moderated functional connectivity

In [2]:
# Load PPI results from analyses that produced Fig. 4A
FC_threat_2sPre = pd.read_csv(root+'PPIs_switchToCheck_Threat_2sPre.csv')
FC_threat_5sPre = pd.read_csv(root+'PPIs_switchToCheck_Threat_5sPre.csv')

In [3]:
earlywin = '0.06s-4.98s'
latewin = '4.98s-9.99s'
df = pd.DataFrame(columns=['Seed','ROI','Peaks','Search_window'])

# Effect of ACC*Check on HB
peaks, sig = getPeakVals(FC_threat_5sPre,'PPI1_seedTCXswitchToCheck','-5s-5s',ROI='HB',seed='ACC_cluster_cope7')
df = addFactorsToDataframe(df, seed='ACC', ROI='HB',peaks=peaks,win='-5s-5s',phase='early')
peaks, sig = getPeakVals(FC_threat_5sPre,'PPI1_seedTCXswitchToCheck','5s-10s',ROI='HB',seed='ACC_cluster_cope7')
df = addFactorsToDataframe(df, seed='ACC', ROI='HB',peaks=peaks,win='5s-10s',phase='late')

# Effect of AI*Check on HB
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI1_seedTCXswitchToCheck',earlywin,ROI='HB',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI', ROI='HB',peaks=peaks,win=earlywin,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI1_seedTCXswitchToCheck',latewin,ROI='HB',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI', ROI='HB',peaks=peaks,win=latewin,phase='late')

# Effect of ACC*Check on DRN
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI1_seedTCXswitchToCheck',earlywin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="ACC_cluster_cope7")
df = addFactorsToDataframe(df, seed='ACC',ROI='DRN',peaks=peaks,win=earlywin,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI1_seedTCXswitchToCheck',latewin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="ACC_cluster_cope7")
df = addFactorsToDataframe(df, seed='ACC',ROI='DRN',peaks=peaks,win=latewin,phase='late')

# Effect of AI*Check on DRN
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI1_seedTCXswitchToCheck',earlywin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI',ROI='DRN',peaks=peaks,win=earlywin,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI1_seedTCXswitchToCheck',latewin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI',ROI='DRN',peaks=peaks,win=latewin,phase='late')

df

/var/folders/ck/bj1v04rj5k97nqvzmysnm2wr0000gn/T/ipykernel_16616/3068014458.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,pd.DataFrame(data={


,Seed,ROI,Peaks,Search_window,Success,Phase,Action
0,ACC,HB,-0.030,-5s-5s,NaN,early,NaN
1,ACC,HB,0.033,-5s-5s,NaN,early,NaN
2,ACC,HB,0.089,-5s-5s,NaN,early,NaN
3,ACC,HB,-0.043,-5s-5s,NaN,early,NaN
4,ACC,HB,0.088,-5s-5s,NaN,early,NaN
...,...,...,...,...,...,...,...
179,AI,DRN,-0.012,4.98s-9.99s,NaN,late,NaN
180,AI,DRN,0.100,4.98s-9.99s,NaN,late,NaN
181,AI,DRN,-0.041,4.98s-9.99s,NaN,late,NaN
182,AI,DRN,0.102,4.98s-9.99s,NaN,late,NaN


In [4]:
# Table S9
print("NOTE: DATA FROM EARLY AND LATE WINDOWS ENTERED")
model = ols('Peaks ~ C(Seed) + C(ROI) + C(Seed)*C(ROI)', data=df).fit()
sm.stats.anova_lm(model, typ=2)

NOTE: DATA FROM EARLY AND LATE WINDOWS ENTERED


,sum_sq,df,F,PR(>F)
C(Seed),0.025,1.000,7.032,0.009
C(ROI),0.004,1.000,1.171,0.281
C(Seed):C(ROI),0.057,1.000,16.128,0.000
Residual,0.633,180.000,NaN,NaN


In [5]:
# Table S8
# Post-hoc comparisons to examine the main effect of Seed
print(df.loc[:,['Seed','Peaks']].groupby('Seed').describe())

print("\nTukey test:")
tukey = pairwise_tukeyhsd(endog=df['Peaks'],
                          groups=df['Seed'],
                          alpha=0.05)
print(tukey)

      Peaks                                              
      count   mean   std    min    25%    50%   75%   max
Seed                                                     
ACC  92.000  0.015 0.066 -0.164 -0.033  0.022 0.054 0.182
AI   92.000 -0.008 0.057 -0.147 -0.042 -0.011 0.027 0.144

Tukey test:
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
   ACC     AI  -0.0232 0.0117 -0.0411 -0.0052   True
----------------------------------------------------


In [6]:
# Main Text
# Descriptive stats for identifying panel that is different according to significant interaction
print(df.loc[:,['Seed','ROI','Peaks']].groupby(['Seed','ROI']).describe())

          Peaks                                              
          count   mean   std    min    25%    50%   75%   max
Seed ROI                                                     
ACC  DRN 46.000 -0.007 0.064 -0.164 -0.060  0.007 0.030 0.133
     HB  46.000  0.037 0.060 -0.074 -0.002  0.039 0.069 0.182
AI   DRN 46.000  0.004 0.059 -0.141 -0.030  0.003 0.042 0.144
     HB  46.000 -0.021 0.053 -0.147 -0.064 -0.020 0.004 0.122


# Fig. 4Bi

2-way ANOVA testing the effects of cortical (ACC/AI) and sub-cortical (DRN/Hb) on the extent to which check switches and time pressure moderated functional connectivity.

In [7]:
earlywin = '0.06s-4.98s'
latewin = '4.98s-9.99s'
df = pd.DataFrame(columns=['Seed','ROI','Peaks','Search_window'])

# Effect of ACC*Check*Threat on HB
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI='HB',seed='ACC_cluster_cope7')
df = addFactorsToDataframe(df, seed='ACC', ROI='HB',peaks=peaks,win=earlywin,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI='HB',seed='ACC_cluster_cope7')
df = addFactorsToDataframe(df, seed='ACC', ROI='HB',peaks=peaks,win=latewin,phase='late')

# Effect of AI*Check*Threat on HB
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI='HB',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI', ROI='HB',peaks=peaks,win=earlywin,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI='HB',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI', ROI='HB',peaks=peaks,win=latewin,phase='late')

# Effect of ACC*Check*Threat on DRN
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="ACC_cluster_cope7")
df = addFactorsToDataframe(df, seed='ACC',ROI='DRN',peaks=peaks,win=earlywin,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="ACC_cluster_cope7")
df = addFactorsToDataframe(df, seed='ACC',ROI='DRN',peaks=peaks,win=latewin,phase='late')

# Effect of AI*Check*Threat on DRN
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI',ROI='DRN',peaks=peaks,win=earlywin,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI='DRN_sub_PAG_edit_sub_SC_edit_ero',seed="insular_cortex_cope7")
df = addFactorsToDataframe(df, seed='AI',ROI='DRN',peaks=peaks,win=latewin,phase='late')

df

/var/folders/ck/bj1v04rj5k97nqvzmysnm2wr0000gn/T/ipykernel_16616/3068014458.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,pd.DataFrame(data={


,Seed,ROI,Peaks,Search_window,Success,Phase,Action
0,ACC,HB,0.022,0.06s-4.98s,NaN,early,NaN
1,ACC,HB,0.064,0.06s-4.98s,NaN,early,NaN
2,ACC,HB,0.057,0.06s-4.98s,NaN,early,NaN
3,ACC,HB,0.060,0.06s-4.98s,NaN,early,NaN
4,ACC,HB,-0.036,0.06s-4.98s,NaN,early,NaN
...,...,...,...,...,...,...,...
179,AI,DRN,0.033,4.98s-9.99s,NaN,late,NaN
180,AI,DRN,0.061,4.98s-9.99s,NaN,late,NaN
181,AI,DRN,0.098,4.98s-9.99s,NaN,late,NaN
182,AI,DRN,0.025,4.98s-9.99s,NaN,late,NaN


In [8]:
# Table S10
print("NOTE: PEAK DATA FROM WHOLE WINDOW ENTERED")
model = ols('Peaks ~ C(Seed) + C(ROI) + C(Seed)*C(ROI)', data=df).fit()
sm.stats.anova_lm(model, typ=2) # Type 2 Anova DataFrame

NOTE: PEAK DATA FROM WHOLE WINDOW ENTERED


,sum_sq,df,F,PR(>F)
C(Seed),0.024,1.000,4.229,0.041
C(ROI),0.062,1.000,11.113,0.001
C(Seed):C(ROI),0.005,1.000,0.850,0.358
Residual,1.000,180.000,NaN,NaN


In [9]:
# Table S11
# Post-hoc comparisons to examine the main effect of sub-cortical ROI
print(df.loc[:,['ROI','Peaks']].groupby('ROI').describe())

print("\nTukey test:")
tukey = pairwise_tukeyhsd(endog=df['Peaks'],
                          groups=df['ROI'],
                          alpha=0.05)
print(tukey)

     Peaks                                              
     count   mean   std    min    25%    50%   75%   max
ROI                                                     
DRN 92.000  0.026 0.073 -0.159 -0.004  0.040 0.069 0.169
HB  92.000 -0.011 0.077 -0.295 -0.058 -0.005 0.033 0.291

Tukey test:
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
   DRN     HB  -0.0366 0.0011 -0.0585 -0.0148   True
----------------------------------------------------


In [11]:
# Table S12
# Post-hoc comparisons to examine the main effect of cortical ROI
print(df.loc[:,['Seed','Peaks']].groupby('Seed').describe())

print("\nTukey test:")
tukey = pairwise_tukeyhsd(endog=df['Peaks'],
                          groups=df['Seed'],
                          alpha=0.05)
print(tukey)

      Peaks                                             
      count   mean   std    min    25%   50%   75%   max
Seed                                                    
ACC  92.000 -0.004 0.083 -0.295 -0.056 0.001 0.051 0.291
AI   92.000  0.019 0.070 -0.184 -0.011 0.031 0.069 0.163

Tukey test:
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
   ACC     AI   0.0226 0.0467 0.0003 0.0449   True
--------------------------------------------------


# Fig. 5A

Two-way ANOVA testing the effects of ROI and behavioral switch on the extent to which time pressure moderated functional connectivity with Hb.

In [12]:
forageSwitch_threat = pd.read_csv(root+'PPIs_switchToForage_Threat_2sPre.csv')

In [13]:
# Read in PPI results related to switch to checking

earlywin = '0.06s-4.98s'
latewin = '4.98s-9.99s'
df = pd.DataFrame(columns=['Seed','ROI','Peaks','Search_window'])
action = "check" # reward

# Effect of HB*Threat*Check on SN
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',seed="HB",peaks=peaks,win=earlywin,action=action,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',seed="HB",peaks=peaks,win=latewin,action=action,phase='late')

# Effect of HB*Threat*Check on VTA
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',seed="HB",peaks=peaks,win=earlywin,action=action,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',seed="HB",peaks=peaks,win=latewin,action=action,phase='late')

# Effect of HB*Threat*Check on DRN
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',seed="HB",peaks=peaks,win=earlywin,action=action,phase='early')
peaks, sig = getPeakVals(FC_threat_2sPre,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',seed="HB",peaks=peaks,win=latewin,action=action,phase='late')

df

/var/folders/ck/bj1v04rj5k97nqvzmysnm2wr0000gn/T/ipykernel_16616/3068014458.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,pd.DataFrame(data={


,Seed,ROI,Peaks,Search_window,Success,Phase,Action
0,HB,SN,0.008,0.06s-4.98s,NaN,early,check
1,HB,SN,0.174,0.06s-4.98s,NaN,early,check
2,HB,SN,0.022,0.06s-4.98s,NaN,early,check
3,HB,SN,-0.042,0.06s-4.98s,NaN,early,check
4,HB,SN,-0.154,0.06s-4.98s,NaN,early,check
...,...,...,...,...,...,...,...
133,HB,DRN,0.014,4.98s-9.99s,NaN,late,check
134,HB,DRN,-0.044,4.98s-9.99s,NaN,late,check
135,HB,DRN,-0.045,4.98s-9.99s,NaN,late,check
136,HB,DRN,0.036,4.98s-9.99s,NaN,late,check


In [14]:
# Read in PPI results related to switch to foraging

# Effect of HB*Reward*Forage on SN
peaks, sig = getPeakVals(forageSwitch_threat,'PPI4_seedTCXswitchToForageXtimepressure',earlywin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',seed="HB",peaks=peaks,win=earlywin,action='forage',phase='early')
peaks, sig = getPeakVals(forageSwitch_threat,'PPI4_seedTCXswitchToForageXtimepressure',latewin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',seed="HB",peaks=peaks,win=latewin,action='forage',phase='late')

# Effect of HB*Reward*Forage on VTA
peaks, sig = getPeakVals(forageSwitch_threat,'PPI4_seedTCXswitchToForageXtimepressure',earlywin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',seed="HB",peaks=peaks,win=earlywin,action='forage',phase='early')
peaks, sig = getPeakVals(forageSwitch_threat,'PPI4_seedTCXswitchToForageXtimepressure',latewin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',seed="HB",peaks=peaks,win=latewin,action='forage',phase='late')

# Effect of HB*Reward*Forage on DRN
peaks, sig = getPeakVals(forageSwitch_threat,'PPI4_seedTCXswitchToForageXtimepressure',earlywin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',seed="HB",peaks=peaks,win=earlywin,action='forage',phase='early')
peaks, sig = getPeakVals(forageSwitch_threat,'PPI4_seedTCXswitchToForageXtimepressure',latewin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',seed="HB",peaks=peaks,win=latewin,action='forage',phase='late')

df

,Seed,ROI,Peaks,Search_window,Success,Phase,Action
0,HB,SN,0.008,0.06s-4.98s,NaN,early,check
1,HB,SN,0.174,0.06s-4.98s,NaN,early,check
2,HB,SN,0.022,0.06s-4.98s,NaN,early,check
3,HB,SN,-0.042,0.06s-4.98s,NaN,early,check
4,HB,SN,-0.154,0.06s-4.98s,NaN,early,check
...,...,...,...,...,...,...,...
271,HB,DRN,-0.014,4.98s-9.99s,NaN,late,forage
272,HB,DRN,0.044,4.98s-9.99s,NaN,late,forage
273,HB,DRN,0.045,4.98s-9.99s,NaN,late,forage
274,HB,DRN,-0.036,4.98s-9.99s,NaN,late,forage


In [16]:
# Table S13
model = ols('Peaks ~ C(ROI) + C(Action) + C(ROI)*C(Action)', data=df).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(ROI),0.000,2.000,0.000,1.000
C(Action),0.020,1.000,4.942,0.027
C(ROI):C(Action),0.022,2.000,2.741,0.066
Residual,1.102,270.000,NaN,NaN


# Fig. 5E

2-way ANOVA testing the effects of ROI and check success on the extent to which time pressure moderated functional connectivity.

In [17]:
successFC_threat = pd.read_csv(root+'PPIs_successfulSwitchToCheck_Threat_2sPre.csv')
failFC_threat = pd.read_csv(root+'PPIs_unsuccessfulSwitchToCheck_Threat_2sPre.csv')

In [18]:
earlywin = '0.06s-4.98s'
latewin = '4.98s-9.99s'
df = pd.DataFrame(columns=['Seed','ROI','Peaks','Search_window'])

# Effect of HB*Threat*Check (success) on SN
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',peaks=peaks,win=earlywin,success='Success',phase='early')
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',peaks=peaks,win=latewin,success='Success',phase='late')

# Effect of HB*Threat*Check (success) on VTA
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',peaks=peaks,win=earlywin,success='Success',phase='early')
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',peaks=peaks,win=latewin,success='Success',phase='late')

# Effect of HB*Threat*Check (success) on DRN
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',peaks=peaks,win=earlywin,success='Success',phase='early')
peaks, sig = getPeakVals(successFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',peaks=peaks,win=latewin,success='Success',phase='late')

# Effect of HB*Threat*Check (fail) on SN
peaks, sig = getPeakVals(failFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',peaks=peaks,win=earlywin,success='Fail',phase='early')
peaks, sig = getPeakVals(failFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='SN',peaks=peaks,win=latewin,success='Fail',phase='late')

# Effect of HB*Threat*Check (fail) on VTA
peaks, sig = getPeakVals(failFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',peaks=peaks,win=earlywin,success='Fail',phase='early')
peaks, sig = getPeakVals(failFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="VTA_Nima_Pauli_sub_SN_Pauli",seed="HB")
df = addFactorsToDataframe(df,ROI='VTA',peaks=peaks,win=latewin,success='Fail',phase='late')

# Effect of HB*Threat*Check (fail) on DRN
peaks, sig = getPeakVals(failFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',earlywin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',peaks=peaks,win=earlywin,success='Fail',phase='early')
peaks, sig = getPeakVals(failFC_threat,'PPI4_seedTCXswitchToCheckXtimepressure',latewin,ROI="DRN_sub_PAG_edit_sub_SC_edit_ero",seed="HB")
df = addFactorsToDataframe(df,ROI='DRN',peaks=peaks,win=latewin,success='Fail',phase='late')

df

/var/folders/ck/bj1v04rj5k97nqvzmysnm2wr0000gn/T/ipykernel_16616/3068014458.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,pd.DataFrame(data={


,Seed,ROI,Peaks,Search_window,Success,Phase,Action
0,NaN,SN,-0.224,0.06s-4.98s,Success,early,NaN
1,NaN,SN,0.031,0.06s-4.98s,Success,early,NaN
2,NaN,SN,0.134,0.06s-4.98s,Success,early,NaN
3,NaN,SN,-0.134,0.06s-4.98s,Success,early,NaN
4,NaN,SN,0.011,0.06s-4.98s,Success,early,NaN
...,...,...,...,...,...,...,...
271,NaN,DRN,0.007,4.98s-9.99s,Fail,late,NaN
272,NaN,DRN,-0.017,4.98s-9.99s,Fail,late,NaN
273,NaN,DRN,-0.087,4.98s-9.99s,Fail,late,NaN
274,NaN,DRN,0.032,4.98s-9.99s,Fail,late,NaN


In [19]:
# Table S15
print("NOTE: PEAK DATA FROM WHOLE WINDOW ENTERED")
model = ols('Peaks ~ C(ROI) + C(Success) + C(ROI)*C(Success)', data=df.loc[df.Phase=='late']).fit()
sm.stats.anova_lm(model, typ=2) # Type 2 Anova DataFrame

NOTE: PEAK DATA FROM WHOLE WINDOW ENTERED


,sum_sq,df,F,PR(>F)
C(ROI),0.007,2.000,0.194,0.824
C(Success),0.406,1.000,23.775,0.000
C(ROI):C(Success),0.008,2.000,0.222,0.801
Residual,2.252,132.000,NaN,NaN
